In [1]:
import torch

In [57]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [58]:
y = x**2
print(y)

tensor([[1., 1.],
        [1., 1.]], grad_fn=<PowBackward0>)


In [59]:
print(x.grad_fn)

None


In [60]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<MulBackward0>) tensor(3., grad_fn=<MeanBackward0>)


In [61]:
a = torch.tensor([1, 2, 3.], requires_grad=True)

In [62]:
out = a.sigmoid()

In [63]:
b = out.sum()

In [66]:
v = torch.tensor(2)
b.backward(v)

In [67]:
print("{} {}".format("a =", a))
print("{} {}".format("out =", out))
print("{} {}".format("b =", b))

a = tensor([1., 2., 3.], requires_grad=True)
out = tensor([0.7311, 0.8808, 0.9526], grad_fn=<SigmoidBackward0>)
b = 2.564429759979248


In [128]:
a = torch.ones(2, 2, requires_grad=True)
b = torch.ones(2, 2, requires_grad=True)
c = a * b
# 添加retain_graph=True这个参数后，才可以进行多次的求导
d = c.sum()
d.backward(retain_graph=True)
print(a.grad)
print(b.grad)
c1 = c * a
c1.sum().backward(retain_graph=True)
print(a.grad)
print(b.grad)

tensor([[1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([[2., 2.],
        [2., 2.]])
tensor([[1., 1.],
        [1., 1.]])


In [44]:
x = torch.randn(3, requires_grad=True)
print(x)

y = x * 2
i = 0
while y.data.norm() < 1000:
    y = y * 2
    i = i + 1
print(y)
print(i)

tensor([ 0.1006, -0.4268,  0.8226], requires_grad=True)
tensor([ 206.0417, -873.9932, 1684.5966], grad_fn=<MulBackward0>)
10


In [47]:
v = torch.tensor([0.1, 1, 1])
y.backward(v, retain_graph=True)
print(x.grad)
x.grad.data.zero_()

tensor([ 204.8000, 2048.0000, 2048.0000])


tensor([0., 0., 0.])

In [71]:
x = torch.ones(1,requires_grad=True)

print(x.data) # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播
y.data *= 5
y.backward()
print(x) # 更改data的值也会影响tensor的值 
print(x.grad)
print(y)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
tensor([10.], grad_fn=<MulBackward0>)


In [72]:
#设置在文件最开始部分
import os
os.environ["CUDA_VISIBLE_DEVICE"] = "1" # 设置默认的显卡

In [78]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, data_dir, info_csv, image_list, transform=None):
        """
        Args:
            data_dir: path to image directory.
            info_csv: path to the csv file containing image indexes
                with corresponding labels.
            image_list: path to the txt file contains image names to training/validation set
            transform: optional transform to be applied on a sample.
        """
        label_info = pd.read_csv(info_csv)
        image_file = open(image_list).readlines()
        self.data_dir = data_dir
        self.image_file = image_file
        self.label_info = label_info
        self.transform = transform

    def __getitem__(self, index):
        """
        Args:
            index: the index of item
        Returns:
            image and its labels
        """
        image_name = self.image_file[index].strip('\n')
        raw_label = self.label_info.loc[self.label_info['Image_index'] == image_name]
        label = raw_label.iloc[:,0]
        image_name = os.path.join(self.data_dir, image_name)
        image = Image.open(image_name).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.image_file)

In [79]:
import torch
from torch import nn
class MLP(nn.Module):
  # 声明带有模型参数的层，这里声明了两个全连接层
  def __init__(self, **kwargs):
    # 调用MLP父类的.__init__()函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
    super(MLP, self).__init__(**kwargs)                 # python2必须写成 super(本类名, self).__init__() ;python3 直接写为 super().__init__()
    self.hidden = nn.Linear(784, 256)
    self.act = nn.ReLU()
    self.output = nn.Linear(256,10)
  # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
  def forward(self, x):
    o = self.act(self.hidden(x))
    return self.output(o)   

In [80]:
X = torch.rand(2,784) # 设置一个随机的输入张量
net = MLP() # 实例化模型
print(net) # 打印模型
net(X) # 前向计算

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1020,  0.1706,  0.0030, -0.0219,  0.3967,  0.1342, -0.1981, -0.0740,
         -0.0516,  0.0757],
        [-0.0637,  0.0903,  0.0661, -0.1102,  0.1871,  0.1439, -0.2156, -0.1051,
          0.0348,  0.0091]], grad_fn=<AddmmBackward0>)

In [81]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
net = MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


In [82]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [5]:
import torch
torch.optim.Adam?